# Kammeröffnungs-Suche mit schichtbasiertem Ansatz

Dieses Notebook implementiert das neue Verfahren zur Erkennung von Kammeröffnungen:
1. Voxelisierung des 3D-Objekts entlang der Anschlussvektor-Achse
2. Berechnung des 2D Boundary (mit dilate)
3. Extraktion aller 2D-Schichten (Stufen) von Stufe 0 bis Max Stufe
4. Später: Binary Search zur Öffnungserkennung

## 1. Imports und Setup

In [ ]:
import os
import json
import numpy as np
import trimesh
import open3d as o3d
import matplotlib.pyplot as plt
import cv2
from scipy.spatial.transform import Rotation
from ipyfilechooser import FileChooser
from IPython.display import display, Image, HTML
import ipywidgets as widgets

# Matplotlib inline für Notebook-Visualisierung
%matplotlib inline

print("✓ Alle Module erfolgreich importiert")

## 2. JSON-Datei auswählen

Wählen Sie die JSON-Datei mit den 3D-Mesh-Daten und Anschlusspunkten.

In [ ]:
# File Chooser für JSON-Datei
json_chooser = FileChooser(
    path='Data',
    filename='',
    title='<b>JSON-Datei mit 3D-Daten auswählen</b>',
    show_hidden=False,
    select_default=False,
    filter_pattern=['*.json']
)

display(json_chooser)

### Ausgewählte Datei anzeigen

In [ ]:
json_file = json_chooser.selected

print(f"JSON-Datei: {json_file}")

if not json_file:
    print("⚠ Bitte wählen Sie eine JSON-Datei aus!")

## 3. Hilfsfunktionen

In [ ]:
def load_json_data(json_file_path):
    """Lädt JSON-Datei im DataSet-Format mit 3D-Mesh-Daten und ConnectionPoints"""
    if not os.path.exists(json_file_path):
        raise FileNotFoundError(f"JSON-Datei nicht gefunden: {json_file_path}")
    
    print(f"Lade JSON-Datei: {json_file_path}")
    
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Validiere erforderliche Felder
    if 'Graphic3d' not in data:
        raise ValueError("JSON-Datei enthält kein 'Graphic3d' Feld!")
    if 'ConnectionPoints' not in data:
        raise ValueError("JSON-Datei enthält kein 'ConnectionPoints' Feld!")
    
    # Lade Mesh-Daten aus Graphic3d
    graphic3d = data['Graphic3d']
    
    if 'Points' not in graphic3d or 'Indices' not in graphic3d:
        raise ValueError("Graphic3d muss 'Points' und 'Indices' enthalten!")
    
    # Konvertiere Points zu Vertices (X, Y, Z -> array)
    points = graphic3d['Points']
    vertices = np.array([[p['X'], p['Y'], p['Z']] for p in points])
    
    # Konvertiere Indices zu Faces (flache Liste -> Triplets)
    indices = graphic3d['Indices']
    if len(indices) % 3 != 0:
        raise ValueError(f"Indices-Anzahl ({len(indices)}) ist nicht durch 3 teilbar!")
    
    faces = np.array(indices).reshape(-1, 3)
    
    # Erstelle Mesh
    mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
    print(f"✓ Mesh geladen: {len(vertices)} Vertices, {len(faces)} Faces")
    
    # Lade ConnectionPoints und konvertiere zu einheitlichem Format
    connection_points = data['ConnectionPoints']
    vectors = []
    
    for cp in connection_points:
        vector = {
            'id': cp['Index'],
            'name': cp.get('Name', f"Point_{cp['Index']}"),
            'position': {
                'x': cp['Point']['X'],
                'y': cp['Point']['Y'],
                'z': cp['Point']['Z']
            },
            'direction': {
                'x': cp['InsertDirection']['X'],
                'y': cp['InsertDirection']['Y'],
                'z': cp['InsertDirection']['Z']
            }
        }
        vectors.append(vector)
    
    print(f"✓ ConnectionPoints geladen: {len(vectors)} Anschlusspunkte")
    
    return mesh, vectors


def rotate_mesh_to_align_vector(mesh, direction_vector, target_direction=np.array([0, 0, 1])):
    """Rotiert das Mesh so, dass der Anschlussvektor in Z-Richtung zeigt"""
    # Konvertiere direction_vector zu numpy array falls nötig
    if isinstance(direction_vector, dict):
        dir_vec = np.array([direction_vector['x'], direction_vector['y'], direction_vector['z']])
    else:
        dir_vec = np.array(direction_vector)
    
    # Normalisiere Vektoren
    dir_vec = dir_vec / np.linalg.norm(dir_vec)
    target_direction = target_direction / np.linalg.norm(target_direction)
    
    # Berechne Rotationsachse
    rotation_axis = np.cross(dir_vec, target_direction)
    rotation_axis_norm = np.linalg.norm(rotation_axis)
    
    if rotation_axis_norm < 1e-6:
        if np.dot(dir_vec, target_direction) > 0:
            print("✓ Anschlussvektor zeigt bereits in Z-Richtung")
            return mesh.copy(), np.eye(4)
        else:
            rotation_axis = np.array([1, 0, 0]) if abs(dir_vec[0]) < 0.9 else np.array([0, 1, 0])
            rotation_angle = np.pi
    else:
        rotation_axis = rotation_axis / rotation_axis_norm
        rotation_angle = np.arccos(np.clip(np.dot(dir_vec, target_direction), -1.0, 1.0))
    
    # Erstelle Rotationsmatrix
    rotation = Rotation.from_rotvec(rotation_angle * rotation_axis)
    rotation_matrix_3x3 = rotation.as_matrix()
    
    rotation_matrix = np.eye(4)
    rotation_matrix[:3, :3] = rotation_matrix_3x3
    
    # Rotiere das Mesh
    rotated_mesh = mesh.copy()
    rotated_mesh.apply_transform(rotation_matrix)
    
    print(f"✓ Mesh rotiert: Winkel = {np.degrees(rotation_angle):.2f}°")
    
    return rotated_mesh, rotation_matrix


def mesh_to_voxels(mesh, voxel_resolution=800):
    """Konvertiert trimesh Mesh zu Open3D Voxel Grid"""
    print(f"Konvertiere Mesh zu Voxeln (Resolution: {voxel_resolution})...")
    
    vertices = np.array(mesh.vertices)
    triangles = np.array(mesh.faces)
    
    o3d_mesh = o3d.geometry.TriangleMesh()
    o3d_mesh.vertices = o3d.utility.Vector3dVector(vertices)
    o3d_mesh.triangles = o3d.utility.Vector3iVector(triangles)
    o3d_mesh.compute_vertex_normals()
    
    bounds = mesh.bounds
    dimensions = bounds[1] - bounds[0]
    max_dimension = np.max(dimensions)
    voxel_size = max_dimension / voxel_resolution
    
    voxel_grid = o3d.geometry.VoxelGrid.create_from_triangle_mesh(o3d_mesh, voxel_size=voxel_size)
    
    print(f"✓ Voxel Grid erstellt: {len(voxel_grid.get_voxels())} Voxel")
    return voxel_grid, voxel_size


def extract_voxel_grid_as_3d_array(voxel_grid):
    """Extrahiert Voxel Grid als 3D numpy array"""
    print("Extrahiere Voxel Grid als 3D Array...")
    
    voxels = voxel_grid.get_voxels()
    if len(voxels) == 0:
        print("⚠ Keine Voxel gefunden!")
        return None, None, None
    
    voxel_coords = []
    for voxel in voxels:
        world_coord = voxel_grid.get_voxel_center_coordinate(voxel.grid_index)
        grid_idx = voxel.grid_index
        voxel_coords.append((*world_coord, *grid_idx))
    
    voxel_coords = np.array(voxel_coords)
    
    world_coords = voxel_coords[:, :3]
    x_min, y_min, z_min = world_coords.min(axis=0)
    x_max, y_max, z_max = world_coords.max(axis=0)
    
    grid_indices = voxel_coords[:, 3:].astype(int)
    ix_min, iy_min, iz_min = grid_indices.min(axis=0)
    ix_max, iy_max, iz_max = grid_indices.max(axis=0)
    
    grid_shape = (ix_max - ix_min + 1, iy_max - iy_min + 1, iz_max - iz_min + 1)
    voxel_array = np.zeros(grid_shape, dtype=bool)
    
    for coord in grid_indices:
        ix, iy, iz = coord - np.array([ix_min, iy_min, iz_min])
        voxel_array[ix, iy, iz] = True
    
    bounds = (x_min, x_max, y_min, y_max, z_min, z_max)
    
    print(f"✓ 3D Array: {grid_shape} (X={grid_shape[0]}, Y={grid_shape[1]}, Z={grid_shape[2]})")
    print(f"  Gefüllte Voxel: {voxel_array.sum()} / {voxel_array.size} ({100*voxel_array.sum()/voxel_array.size:.2f}%)")
    
    return voxel_array, bounds, grid_shape


def create_2d_boundary(voxel_array, dilate_iterations=2, kernel_size=3):
    """Erstellt 2D Boundary durch Projektion und morphologische Operationen"""
    print("Erstelle 2D Boundary...")
    
    projection = np.any(voxel_array, axis=2)
    projection_image = (projection * 255).astype(np.uint8)
    
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    dilated = cv2.dilate(projection_image, kernel, iterations=dilate_iterations)
    
    grad_x = cv2.Sobel(dilated, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(dilated, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)
    
    threshold = gradient_magnitude.max() * 0.3
    boundary_image = (gradient_magnitude > threshold).astype(np.uint8) * 255
    
    print(f"✓ Boundary erstellt: {(boundary_image > 0).sum()} Boundary-Pixel")
    
    return boundary_image, projection_image


def find_connection_point_z_level(connection_point, bounds, grid_shape, voxel_size):
    """Findet die Z-Stufe des Anschlusspunkts"""
    x_min, x_max, y_min, y_max, z_min, z_max = bounds
    
    z_world = connection_point['z']
    z_normalized = (z_world - z_min) / (z_max - z_min)
    z_level = int(z_normalized * (grid_shape[2] - 1))
    z_level = max(0, min(grid_shape[2] - 1, z_level))
    
    print(f"✓ Anschlusspunkt Z-Stufe: {z_level} / {grid_shape[2] - 1}")
    
    return z_level


def extract_2d_layer(voxel_array, z_level):
    """Extrahiert eine 2D-Schicht aus dem 3D Voxel Array"""
    if z_level < 0 or z_level >= voxel_array.shape[2]:
        return None
    
    layer = voxel_array[:, :, z_level]
    layer_image = (layer * 255).astype(np.uint8)
    
    return layer_image

print("✓ Hilfsfunktionen definiert")

## 4. JSON-Datei laden und Anschlusspunkt auswählen

In [ ]:
# Lade JSON-Datei
mesh, vectors = load_json_data(json_file)

# Zeige verfügbare Anschlusspunkte
if vectors:
    print("\nVerfügbare Anschlusspunkte:")
    for v in vectors:
        print(f"  P{v['id']}: Position ({v['position']['x']:.4f}, {v['position']['y']:.4f}, {v['position']['z']:.4f})")
else:
    print("⚠ Keine Anschlusspunkte gefunden!")

### Anschlusspunkt-ID auswählen

In [ ]:
# Dropdown für Anschlusspunkt-Auswahl
if vectors:
    connection_point_dropdown = widgets.Dropdown(
        options=[(f"P{v['id']} - Position ({v['position']['x']:.4f}, {v['position']['y']:.4f}, {v['position']['z']:.4f})", v['id']) for v in vectors],
        description='Anschlusspunkt:',
        style={'description_width': 'initial'}
    )
    display(connection_point_dropdown)
else:
    print("⚠ Keine Anschlusspunkte verfügbar")

## 5. Verarbeitung starten

In [ ]:
# Wähle Anschlusspunkt
connection_point_id = connection_point_dropdown.value
vector = next((v for v in vectors if v['id'] == connection_point_id), None)

if vector is None:
    print(f"⚠ Anschlusspunkt mit ID {connection_point_id} nicht gefunden!")
else:
    print(f"\n{'='*60}")
    print(f"Verarbeite Anschlusspunkt P{vector['id']}")
    print(f"{'='*60}")
    print(f"Position: ({vector['position']['x']:.6f}, {vector['position']['y']:.6f}, {vector['position']['z']:.6f})")
    print(f"Richtung: ({vector['direction']['x']:.6f}, {vector['direction']['y']:.6f}, {vector['direction']['z']:.6f})")
    
    # Rotiere Mesh
    rotated_mesh, rotation_matrix = rotate_mesh_to_align_vector(mesh, vector['direction'])
    
    # Rotiere Anschlusspunkt
    connection_point_world = np.array([
        vector['position']['x'],
        vector['position']['y'],
        vector['position']['z'],
        1.0
    ])
    rotated_connection_point = rotation_matrix @ connection_point_world
    rotated_connection_dict = {
        'x': rotated_connection_point[0],
        'y': rotated_connection_point[1],
        'z': rotated_connection_point[2]
    }
    
    print(f"Rotierter Anschlusspunkt: ({rotated_connection_dict['x']:.6f}, {rotated_connection_dict['y']:.6f}, {rotated_connection_dict['z']:.6f})")
    
    # Voxelisierung
    voxel_grid, voxel_size = mesh_to_voxels(rotated_mesh, voxel_resolution=800)
    
    # Extrahiere 3D Array
    voxel_array, bounds, grid_shape = extract_voxel_grid_as_3d_array(voxel_grid)
    
    if voxel_array is not None:
        # Erstelle Ausgabe-Ordner
        json_name = os.path.splitext(os.path.basename(json_file))[0]
        output_dir = os.path.join("Data", f"{json_name}_P{vector['id']}_search")
        os.makedirs(output_dir, exist_ok=True)
        print(f"\n✓ Ausgabe-Ordner: {output_dir}")
        
        # Erstelle 2D Boundary
        boundary_image, projection_image = create_2d_boundary(voxel_array)
        
        # Speichere Bilder
        boundary_filename = os.path.join(output_dir, f"{json_name}_P{vector['id']}_boundary.png")
        projection_filename = os.path.join(output_dir, f"{json_name}_P{vector['id']}_projection.png")
        
        cv2.imwrite(boundary_filename, boundary_image)
        cv2.imwrite(projection_filename, projection_image)
        
        print(f"✓ 2D Boundary gespeichert: {boundary_filename}")
        print(f"✓ 2D Projektion gespeichert: {projection_filename}")
        
        # Finde Z-Stufe
        z_start = find_connection_point_z_level(rotated_connection_dict, bounds, grid_shape, voxel_size)
        z_end = grid_shape[2] - 1
        
        print(f"\nZ-Bereich: Stufe {z_start} (Anschlusspunkt) bis {z_end} (Max)")
        print(f"Anzahl Stufen: {z_end - z_start + 1}")
    else:
        print("⚠ Fehler beim Erstellen des Voxel Arrays!")

## 6. Visualisierung: 2D Boundary und Projektion

In [ ]:
# Zeige Boundary und Projektion
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].imshow(projection_image, cmap='gray', origin='lower')
axes[0].set_title(f'2D Projektion (Max-Projektion in Z)', fontsize=14, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(boundary_image, cmap='hot', origin='lower')
axes[1].set_title(f'2D Boundary (Dilate + Gradient)', fontsize=14, fontweight='bold')
axes[1].axis('off')

plt.tight_layout()
plt.show()

## 7. Layer-Extraktion und Speicherung

In [ ]:
# Speichere alle Layer-Bilder
layers_dir = os.path.join(output_dir, "layers")
os.makedirs(layers_dir, exist_ok=True)

saved_files = []

print(f"Speichere alle Layer von Stufe {z_start} bis {z_end}...")

for z_level in range(z_start, z_end + 1):
    layer_image = extract_2d_layer(voxel_array, z_level)
    
    if layer_image is not None:
        filename = os.path.join(layers_dir, f"{json_name}_P{vector['id']}_layer_{z_level:04d}.png")
        cv2.imwrite(filename, layer_image)
        saved_files.append(filename)

print(f"\n✓ Gespeichert: {len(saved_files)} Layer-Bilder in {layers_dir}")

## 8. Layer-Übersicht (16 Samples)

In [ ]:
# Wähle 16 gleichmäßig verteilte Samples
sample_count = 16
total_layers = z_end - z_start + 1

if total_layers < sample_count:
    sample_count = total_layers

sample_indices = np.linspace(z_start, z_end, sample_count, dtype=int)

# Erstelle 4x4 Grid
fig, axes = plt.subplots(4, 4, figsize=(16, 16))
axes = axes.flatten()

for idx, z_level in enumerate(sample_indices):
    layer_image = extract_2d_layer(voxel_array, z_level)
    
    axes[idx].imshow(layer_image, cmap='gray', origin='lower')
    axes[idx].set_title(f'Stufe {z_level}', fontsize=10, fontweight='bold')
    axes[idx].axis('off')

# Deaktiviere übrige Subplots
for idx in range(len(sample_indices), 16):
    axes[idx].axis('off')

plt.suptitle(f'Layer-Übersicht: {sample_count} Samples von Stufe {z_start} bis {z_end}', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()

# Speichere Übersicht
overview_filename = os.path.join(output_dir, f"{json_name}_P{vector['id']}_layers_overview.png")
plt.savefig(overview_filename, dpi=150, bbox_inches='tight')
print(f"✓ Layer-Übersicht gespeichert: {overview_filename}")

plt.show()

## 9. Interaktive Layer-Anzeige

In [ ]:
# Slider für interaktive Layer-Auswahl
layer_slider = widgets.IntSlider(
    value=z_start,
    min=z_start,
    max=z_end,
    step=1,
    description='Z-Stufe:',
    continuous_update=False,
    style={'description_width': 'initial'}
)

output_widget = widgets.Output()

def update_layer(change):
    with output_widget:
        output_widget.clear_output(wait=True)
        
        z_level = change['new']
        layer_image = extract_2d_layer(voxel_array, z_level)
        
        if layer_image is not None:
            fig, ax = plt.subplots(figsize=(10, 10))
            ax.imshow(layer_image, cmap='gray', origin='lower')
            ax.set_title(f'Layer {z_level} / {z_end} (Abstand von Anschlusspunkt: {z_level - z_start} Stufen)', 
                        fontsize=14, fontweight='bold')
            ax.axis('off')
            plt.tight_layout()
            plt.show()
        else:
            print(f"⚠ Layer {z_level} konnte nicht geladen werden")

layer_slider.observe(update_layer, names='value')

display(layer_slider)
display(output_widget)

# Zeige initialen Layer
update_layer({'new': z_start})

## 10. Zusammenfassung

In [ ]:
print("\n" + "="*60)
print("ZUSAMMENFASSUNG")
print("="*60)
print(f"JSON-Datei: {os.path.basename(json_file)}")
print(f"Anschlusspunkt: P{vector['id']}")
print(f"\nMesh-Daten:")
print(f"  Vertices: {len(mesh.vertices)}")
print(f"  Faces: {len(mesh.faces)}")
print(f"\nVoxel-Auflösung: 800")
print(f"Grid-Shape: {grid_shape}")
print(f"Voxel-Size: {voxel_size:.6f}")
print(f"\nZ-Stufe (Anschlusspunkt): {z_start}")
print(f"Z-Stufe (Maximum): {z_end}")
print(f"Anzahl Stufen: {z_end - z_start + 1}")
print(f"\nGespeicherte Layer-Bilder: {len(saved_files)}")
print(f"Ausgabe-Ordner: {output_dir}")
print("="*60)
print("✓ Verarbeitung erfolgreich abgeschlossen!")
print("\nNächste Schritte:")
print("  - Binary Search Implementierung für Kammeröffnungs-Erkennung")
print("  - Analyse der Layer-Sequenz zur Öffnungsdetektion")